In [0]:
# Importações

from pyspark.sql.functions import (col, sum as sum_, countDistinct, avg, min as min_, max as max_)

In [0]:
# DataFrame

clientes_df = spark.table("ecommerce.silver.clientes")
pedidos_df = spark.table("ecommerce.silver.pedidos")
avaliacoes_df = spark.table("ecommerce.silver.avaliacoes")

In [0]:
# Agregação de pedidos por cliente

pedidos_agg_df = (
    pedidos_df
    .groupBy("cliente_id")
    .agg(
        sum_("valor_liquido").alias("total_gasto"),
        countDistinct("pedido_id").alias("quantidade_pedidos"),
        avg("valor_liquido").alias("ticket_medio"),
        min_("data_pedido").alias("data_primeiro_pedido"),
        max_("data_pedido").alias("data_ultimo_pedido")
    )
)

# Agregação de produtos avaliados

avaliacoes_agg_df = (
    avaliacoes_df
    .groupBy("cliente_id")
    .agg(
        countDistinct("produto_id").alias("produtos_avaliados")
    )
)

# Join final (base = clientes)

c = clientes_df.alias("c")
p = pedidos_agg_df.alias("p")
a = avaliacoes_agg_df.alias("a")

clientes_metricas_df = (
    c
    .join(p, "cliente_id", "left")
    .join(a, "cliente_id", "left")
    .select(
        col("cliente_id"),
        col("nome"),
        col("cidade"),
        col("estado"),
        col("regiao"),

        col("total_gasto"),
        col("quantidade_pedidos"),
        col("ticket_medio"),
        col("data_primeiro_pedido"),
        col("data_ultimo_pedido"),
        col("produtos_avaliados")
    )
)

In [0]:
#Salvando DataFrame na camada silver
clientes_metricas_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.clientes_metricas")